**Using a pre-trained convnet to feature extraction**


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

local_zip = 'cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [ ]:
import tensorflow.keras

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'data/cats_and_dogs_filtered/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
# test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
# test_features, test_labels = extract_features(test_dir, 1000)

In [ ]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
# test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
print (train_features.shape)
print (train_features[1])

In [ ]:
dist = np.sqrt(np.sum(np.square(np.subtract(train_features[18], train_features[24]))))

In [ ]:
print (dist)

Referência: François Chollet. Deep Learning with Python. November 2017  

**Mais um exemplo (uma imagem por vez)**

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2

path1 = os.path.join(base_dir, 'validation/dogs/dog.2010.jpg')
img1 = cv2.imread(path1) 
cv2_imshow(img1)


path2 = os.path.join(base_dir, 'validation/dogs/dog.2011.jpg')
img2 = cv2.imread(path2) 
cv2_imshow(img2)

In [ ]:
print(img1.shape)
print(img2.shape)

Vamos carregar a imagem usando PIL (biblioteca para processamento de imagens em Python)

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from PIL import Image

img1 = load_img(path1, target_size=(150, 150))
img2 = load_img(path2, target_size=(150, 150))

Rescale de cada pixels de (0 a 255) para [0, 1]

In [ ]:
from numpy import asarray

## img1

img1 = asarray(img1)

print('Data Type: %s' % img1.dtype)
print('Min: %.3f, Max: %.3f' % (img1.min(), img1.max()))

img1 = img1.astype('float32')
img1 /= 255.0

print('Min: %.3f, Max: %.3f' % (img1.min(), img1.max()))

### img2

img2 = asarray(img2)

print('Data Type: %s' % img2.dtype)
print('Min: %.3f, Max: %.3f' % (img2.min(), img2.max()))

img2 = img2.astype('float32')
img2 /= 255.0

print('Min: %.3f, Max: %.3f' % (img2.min(), img2.max()))


In [ ]:
x = img_to_array(img1)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(x.shape)

y = img_to_array(img2)  # this is a Numpy array with shape (3, 150, 150)
y = y.reshape((1,) + y.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

print(y.shape)

In [ ]:
features_x = conv_base.predict(x)
features_y = conv_base.predict(y)

print(features_x.shape)
print(features_x)

In [ ]:
features_x = np.reshape(features_x, (1, 4 * 4 * 512))
features_y = np.reshape(features_y, (1, 4 * 4 * 512))

In [ ]:
print(features_x.shape)

In [ ]:
dist = np.sqrt(np.sum(np.square(np.subtract(features_x, features_y))))

In [ ]:
print(dist)